
# RANDY


Author: PVH
Date Created: Feb. 15th, 2018


### Description

This notebook is intended to demonstrate the capabilities of RANDY to help organize experiments



#### Let's start with something simple, let's load some classes corresponding to samples.

In [1]:
# homegrown libraries
from sample.sample import Sample
from sample.matter import Matter
from sample.sequence import Sequence
from sample.cell import Cell
from sample.solute import Solute
from sample.solution import Solution
from sample.buffer import Buffer

print 'Complete!'

Complete!


Each of these modules correspond to different objects that you can interact with throughout experiments.

### Making a buffer by hand

First, let's make a solution- something simple like elution buffer. To initialize each of these elements, just enter the object name, with arguments for its name, and anything else you might find important:

In [2]:
# initialize a solute and a solution (Tris)
solute   = Solute(name='Tris-Cl',concentration='100 mM')
solution = Solution(name='dH2O',volume='500 mL')

# mix the two together to make a buffer
my_buffer = solute + solution

# label the buffer
my_buffer.name = 'Buffer EB'

# print out object
print my_buffer

          -- Sample object --           
            Name: Buffer EB             
                Owner:                  
             Volume: 0.5 L              
              Container:                
                                        
               Contents:                
----------------------------------------
            Solute(Tris-Cl)             
             Solution(dH2O)             


Well, this might get pretty tedious if everytime we want to create buffers or mixtures we have to list all the inputs. Is there a better way to do this?

### Cloud-based database

Absolutely. The system here has been designed to have a central repository in a lab based in the cloud (specific, Google Drive). To load one of these, make a new database object:

In [3]:
# import the database module
from database.database import Database

db = Database() # create a database object, loaded with local data
db.sync_local() # sync local files with the cloud data


Database loaded!
Database - sequences sync'ed (0 updated, 0 added, 2 total)
Database - cells sync'ed (0 updated, 0 added, 1 total)
Database - solutions sync'ed (0 updated, 0 added, 2 total)
Database - solutes sync'ed (0 updated, 0 added, 1 total)
Database - buffers sync'ed (1 updated, 0 added, 2 total)
Database - elements sync'ed (0 updated, 0 added, 0 total)


#### Cool! Now let's make elution buffer in a much quicker way:

In [4]:
# load buffer EB from the database
buffer1  = db.load('Buffer EB')

# print out object
print buffer1

          -- Buffer object --           
            Name: Buffer EB             
                Owner:                  
             Volume: 0.5 L              
           Instructions: mix            
Contents: (Solution(dH2O),Solute(Tris-Cl))


Wait- this looks a little different? It's ok. For the purpose of brevity, you can call objects as buffers instead of solutes and solutions all day. Don't believe me? Make some cells and add them to the solution:

In [5]:
# make a cell object
cells = Cell(name='DH5a',species='E.Coli',count=1e6)
sample_w_buffer = buffer1 + cells

# label the buffer
sample_w_buffer.name = 'Cells in elution buffer'

# print out object
print sample_w_buffer
sample_w_buffer.display_contents()

          -- Sample object --           
     Name: Cells in elution buffer      
                Owner:                  
             Volume: 0.5 L              
              Container:                
                                        
               Contents:                
----------------------------------------
           Buffer(Buffer EB)            
               Cell(DH5a)               
Component 1:
          -- Buffer object --           
            Name: Buffer EB             
                Owner:                  
             Volume: 0.5 L              
           Instructions: mix            
Contents: (Solution(dH2O),Solute(Tris-Cl))
Component 2:
           -- Cell object --            
               Name: DH5a               
                Owner:                  
            Species: E.Coli             
            Count: 1000000.0            


That looks much cleaner.

### Adjusting volumes

You may have noticed that the buffer has a default volume. Well, this is what the recipe dictates, but it can change depending on what you want to do. Here, we result the sample volume:

In [6]:
sample_w_buffer.volume = '100 mL'
sample_w_buffer.display_contents()

Component 1:
          -- Buffer object --           
            Name: Buffer EB             
                Owner:                  
            Volume: 100.0 mL            
           Instructions: mix            
Contents: (Solution(dH2O),Solute(Tris-Cl))
Component 2:
           -- Cell object --            
               Name: DH5a               
                Owner:                  
            Species: E.Coli             
            Count: 200000.0             


## Making sequence objects

So far, we have only talked about basic buffers and cells. What if we have something more complicated, such as DNA sequences? We can instance those aswell:

In [7]:
plasmid = Sequence(name='MyPlasmid',mass='1 ug',sequence='ATCGGAGACTAGGCCATAGC',material='dsDNA')
buffer1  = db.load('Buffer EB',volume='50 uL')

sample = plasmid + buffer1
sample.display_contents()

Component 1:
         -- Sequence object --          
            Name: MyPlasmid             
                Owner:                  
  Molecular_weight: 12.3948926 kg/mol   
              Mass: 1.0 ug              
       Moles: 80.6783916788 pmol        
        Concentration: 20.0 mg/L        
     Molarity: 1.61356783358 umol/L     
            Material: dsDNA             
         Total_volume: 50.0 uL          

               Sequence:                
1         11        
ATCGGAGACTAGGCCATAGC

Component 2:
          -- Buffer object --           
            Name: Buffer EB             
                Owner:                  
            Volume: 50.0 uL             
           Instructions: mix            
Contents: (Solution(dH2O),Solute(Tris-Cl))
